## <center>Customer Personality Analysis</center>

In [202]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [203]:
# Importing libraries
import pandas as pd
import numpy as np

In [204]:
# Reading the input data file
import csv
df_camp = pd.read_csv('../input/customer-personality-analysis/marketing_campaign.csv', sep='\t')
df_camp.head()

In [205]:
# Describing the input dataframe
df_camp.describe()

In [206]:
# Null check in dataframe
df_camp.isna().sum()

In [207]:
# Remove the columns where Income is null
df_camp = df_camp[~df_camp['Income'].isna()]
df_camp.isna().sum()

In [208]:
# Dropping Z variable because it is already considered
df_camp.drop(['Z_CostContact','Z_Revenue'],axis=1 ,inplace=True)
df_camp.columns

In [209]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.displot(df_camp['Year_Birth'])

In [210]:
# Handling education column
df_camp['Education'] = df_camp['Education'].replace(['PhD','Master','2n Cycle'],'Post Graduate')
df_camp['Education'] = df_camp['Education'].replace(['Graduation','Basic'],'Under Graduate')
df_camp['Education'].unique()

In [211]:
# Vizualizing distribution of education column
sns.displot(df_camp['Education'])

In [212]:
## Vizualizing distribution of marital status column
sns.displot(df_camp['Marital_Status'])
plt.xticks(rotation=45)
plt.show()

In [213]:
df_camp['Marital_Status'] = df_camp['Marital_Status'].replace(['Together','Married'],'Relationship')
df_camp['Marital_Status'] = df_camp['Marital_Status'].replace(['Single','Divorced','Widow','Alone','Absurd','YOLO'],'Single')
df_camp['Marital_Status'].unique()

In [214]:
# Vizualizing distribution of updated marital status column
sns.displot(df_camp['Marital_Status'])

In [215]:
# Combining features
print(df_camp['Kidhome'].unique())
print(df_camp['Teenhome'].unique())

df_camp['Total_kids'] = df_camp['Kidhome'] + df_camp['Teenhome']
df_camp['Total_kids'].unique()

In [216]:
# Vizualizing distribution of total kids column
sns.displot(df_camp['Total_kids'].sort_values().astype(str))

In [217]:
# Viewing all the columns in the new dataframe
df_camp.columns

In [218]:
# Creating new features using expense
df_camp['Total_expense'] = df_camp['MntWines'] + df_camp['MntFruits'] + df_camp['MntMeatProducts'] + df_camp['MntFishProducts'] + df_camp['MntSweetProducts'] + df_camp['MntGoldProds']
sns.displot(df_camp['Total_expense'])

In [219]:
# Creating new features using campaign acceptance features
df_camp['Total_accepted'] = df_camp['AcceptedCmp1'] + df_camp['AcceptedCmp2'] + df_camp['AcceptedCmp3'] + df_camp['AcceptedCmp4'] + df_camp['AcceptedCmp5']
df_camp['Total_accepted'].unique()

In [220]:
# Vizualizing the distribution plot for total accepted feature
sns.displot(df_camp['Total_accepted'].astype(str))

In [221]:
# Creating new feature - Total purchase
df_camp['Total_purchase'] = df_camp['NumDealsPurchases'] + df_camp['NumWebPurchases'] + df_camp['NumCatalogPurchases'] + df_camp['NumStorePurchases'] 
sns.displot(df_camp['Total_purchase'])

In [222]:
df_camp['Dt_Customer'] = pd.to_datetime(df_camp['Dt_Customer'])

In [223]:
df_camp['first_date'] = pd.to_datetime('01-01-2015')
df_camp['days_engaged'] = (df_camp['first_date'] - df_camp['Dt_Customer']).dt.days

In [224]:
# importing date class from datetime module
from datetime import date

# creating the date object of today's date
todays_date = date.today()

# fetching the current year, month and day of today
df_camp['current_year'] = todays_date.year

df_camp['total_year_enrol'] = df_camp['current_year'] - pd.DatetimeIndex(df_camp['Dt_Customer']).year

In [225]:
df_camp.columns

In [226]:
# Dropping unnecessary columns
df_camp.drop(['Kidhome','Teenhome', 'Dt_Customer', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'first_date','ID'],axis=1,inplace=True)
df_camp.columns

In [227]:
# Getting all the categorical variable
catg_col=[]
for i in df_camp.columns:
    if(df_camp[i].dtype =='object'):
        catg_col.append(i)
        
print(catg_col)

In [228]:
# Label encoding for all categorical nominal variable
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
for i in catg_col:
    df_camp[i]= encoder.fit_transform(df_camp[i])
    
print(df_camp)

In [229]:
# Implement standardization of features
from sklearn.preprocessing import StandardScaler
scale_data = StandardScaler()
df_val = scale_data.fit_transform(df_camp.values)
df_model = pd.DataFrame(df_val, columns=df_camp.columns)
df_model

In [230]:
# Determining correlation between all correlation
plt.figure(figsize=(15,15))
sns.heatmap(df_camp.corr(),annot=True,cmap = 'YlGnBu',linewidths=1)

In [231]:
# Creating new dataframe for clustering
df_model.columns
X = df_model.iloc[:,[3,10]]

In [232]:
# Elbow method to calculate N
from sklearn.cluster import KMeans
wcss = [] 
for i in range(1, 11): 
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(X) 
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.xlabel('Number of clusters')
plt.ylabel('WCSS') 
plt.show()

In [233]:
# Validating all the columns which will be used for clustering
X.columns

In [234]:
# K means Clustering
kmeans = KMeans(n_clusters=2, init="k-means++",random_state=50)
label = kmeans.fit_predict(X[['Income', 'Total_purchase']])

In [235]:
#Getting the Centroids
centroids = kmeans.cluster_centers_
u_labels = np.unique(label)
 
#plotting the results:
for i in u_labels:
    plt.scatter(X[label == i].iloc[:,0] , X[label == i].iloc[:,1] , label = i)
plt.scatter(centroids[:,0] , centroids[:,1] , s = 60, color = "black")
plt.xlabel('Income')
plt.ylabel('Total Purchase')
plt.legend()
plt.show()

## Insights

The most important insight is that customer who have low income has less purchases are much wiser than customers who have low income and high purchase